# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [3]:
import pandas as pd
import numpy as np
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [4]:
stocks = pd.read_csv("sp_500_stocks.csv")

stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [5]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [6]:
ticker = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{ticker}/quote/?token={IEX_CLOUD_API_TOKEN}" # ? for vars in api endpoints

# making the GET request

response = requests.get(api_url)
# check for response code 
response.status_code

200

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [7]:
# parsing data
data = response.json()
# getting two relevant attributes
price = data["latestPrice"]
market_cap = data["marketCap"]

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [8]:
# listing the columns we want to add to our data frame
columns = ["Ticker", "Stock Price", "Market Capitalization", "Numbers of Shares to Buy"]
# creating df
traiding_df = pd.DataFrame(columns=columns)
traiding_df

,Ticker,Stock Price,Market Capitalization,Numbers of Shares to Buy


In [9]:
# appending data to df 
traiding_df.append(
    pd.Series([
        ticker, 
        price, # Stock Price
        market_cap, # Market Capitalization
        "N/A" # buffer
    ],
    index=columns
    ), 
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Numbers of Shares to Buy
0,AAPL,175.9,2884242739206,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [10]:
# Executing the HTTP req for each stock in the stocks data frame (very slow)
for stock in stocks["Ticker"][:5]: # testing for first 5 stocks
    api_url = f"https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}" # ? for vars in api endpoints
    data = requests.get(api_url).json()
    # adding data to dataframe
    traiding_df = traiding_df.append(
    pd.Series([
        stock, 
        data["latestPrice"], # Stock Price
        data["marketCap"], # Stock Price
        "N/A" # buffer
    ],
    index=columns
    ), 
    ignore_index=True
)


In [11]:
traiding_df

,Ticker,Stock Price,Market Capitalization,Numbers of Shares to Buy
0,A,154.040,47157872720,N/A
1,AAL,18.661,12403530925,N/A
2,AAP,237.430,14260581378,N/A
3,AAPL,172.500,2820439551787,N/A
4,ABBV,133.310,237094099754,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [12]:
# function wich splits a list into sub-lists of size n
def chunks(list, n):
    for i in range(0, len(list), n):
        yield list[i: i + n]   

In [13]:
# getting the group of tickers from df
symbol_groups = list(chunks(stocks["Ticker"], 100))
# looping through each series of this list and append tickers (comma seperated string of 100) to a list 
symbol_strings = []
for i in range(len(symbol_groups)):
    symbol_strings.append(",".join(symbol_groups[i]))
    # print(symbol_strings[i])
# looping through each symbol string and make a batch api call
for symbol_string in symbol_strings:
    # creating the batch api url
    params = f"batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    batch_api_call_url = "https://sandbox.iexapis.com/stable/stock/market/" + params 
    # making the call
    data = requests.get(batch_api_call_url).json()
    # parsing data for every stock and adding it to df
    for symbol in symbol_string.split(","):
        traiding_df = traiding_df.append(
            pd.Series([
                symbol,
                data[symbol]["quote"]["latestPrice"],
                data[symbol]["quote"]["marketCap"],
                "N/A"
            ], index=columns
            ), ignore_index=True
        )
traiding_df

,Ticker,Stock Price,Market Capitalization,Numbers of Shares to Buy
0,A,154.040,47157872720,N/A
1,AAL,18.661,12403530925,N/A
2,AAP,237.430,14260581378,N/A
3,AAPL,172.500,2820439551787,N/A
4,ABBV,133.310,237094099754,N/A
...,...,...,...,...
505,YUM,134.540,39813373985,N/A
506,ZBH,125.780,26179621541,N/A
507,ZBRA,592.700,32101268970,N/A
508,ZION,63.550,9563143992,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [14]:
# getting the portolio size w/ str handling
try:
    portfolio = float(input("Please enter your Portfolio size"))
except ValueError:
    print("This is not a number!")
    portfolio = float(input("Please enter your Portfolio size"))
portfolio

1000000.0

In [15]:
# looping through each stock and add the number of shares to by
for i in range(len(traiding_df.index)):
    # calculating number of shares to buy for each stock
    n_shares_to_buy = math.floor(portfolio / traiding_df.loc[i, "Stock Price"])
    # adding to "Numbers of shares to buy" column
    traiding_df.loc[i, "Numbers of Shares to Buy"] = n_shares_to_buy
traiding_df

,Ticker,Stock Price,Market Capitalization,Numbers of Shares to Buy
0,A,154.040,47157872720,6491
1,AAL,18.661,12403530925,53587
2,AAP,237.430,14260581378,4211
3,AAPL,172.500,2820439551787,5797
4,ABBV,133.310,237094099754,7501
...,...,...,...,...
505,YUM,134.540,39813373985,7432
506,ZBH,125.780,26179621541,7950
507,ZBRA,592.700,32101268970,1687
508,ZION,63.550,9563143992,15735


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: